In [2]:
""" # Instalar a vesão 3.0.3 do PySpark
!pip install pyspark==3.0.3

# Instalar o NGROK
!wget -qnc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -n -q ngrok-stable-linux-amd64.zip
 """
# Autenticar a sessão do SparkUI com NGROK
!./ngrok authtoken 2KBeQEmmd1YNlQ86GGKf3KFOkb3_6sQH7JEnvEhDxwn9A7WnT
get_ipython().system_raw('./ngrok http 4050 &')
!sleep 10
!curl -s http://localhost:4040/api/tunnels | grep -Po 'public_url":"(?=https)\K[^"]*'

""" from google.colab import drive
drive.mount('/content/drive') """

/bin/bash: line 1: ./ngrok: No such file or directory


/bin/bash: line 1: ./ngrok: No such file or directory


" from google.colab import drive\ndrive.mount('/content/drive') "

In [7]:
""" !pip install pyspark
!pip install pyarrow """
#!pip install pydeequ
#!pip install pyspark
import os

os.environ['SPARK_VERSION'] = '3.3.0'



In [5]:
import pyspark
print(pyspark.__version__)

3.3.0


In [8]:
from pyspark.sql import SparkSession

import pydeequ

spark = (
    SparkSession.builder                  
      .config('spark.ui.port', '4050')
      .config("spark.jars.packages", pydeequ.deequ_maven_coord)
      .config("spark.jars.excludes", pydeequ.f2j_maven_coord)
      .appName("SparkSQL")
      .getOrCreate()
)


23/10/24 23:32:49 WARN Utils: Your hostname, valdir-G5-5590 resolves to a loopback address: 127.0.1.1; using 192.168.0.43 instead (on interface wlp4s0)
23/10/24 23:32:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/valdir/Documentos/CURSO_DNC/novo-curso-dnc/dnc/lib/python3.8/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/valdir/.ivy2/cache
The jars for the packages stored in: /home/valdir/.ivy2/jars
com.amazon.deequ#deequ added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-5240a942-c3c9-41d9-b1a2-431c9d4178db;1.0
	confs: [default]
	found com.amazon.deequ#deequ;2.0.3-spark-3.3 in central
	found org.scala-lang#scala-reflect;2.12.10 in central
	found org.scalanlp#breeze_2.12;0.13.2 in central
	found org.scalanlp#breeze-macros_2.12;0.13.2 in central
	found com.github.fommil.netlib#core;1.1.2 in central
	found net.sf.opencsv#opencsv;2.3 in central
	found com.github.rwl#jtransforms;2.4.0 in central
	found junit#junit;4.8.2 in central
	found org.apache.commons#commons-math3;3.2 in central
	found org.spire-math#spire_2.12;0.13.0 in central
	found org.spire-math#spire-macros_2.12;0.13.0 in central
	found org.typelevel#machinist_2.12;0.6.1 in central
	found com.chuusai#shapeless_2.12;2.3.2 in central
	found org.typelevel#macro-compat_2.12;1.1.1 

23/10/24 23:33:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [11]:
from pyspark.sql.types import StructType, StructField, IntegerType, DoubleType, StringType, TimestampType
from pyspark.sql.functions import col

schema_remetente_destinatario = StructType([
    StructField('nome', StringType()),
    StructField('banco', StringType()),
    StructField('tipo', StringType())
])

schema_base_pix = StructType([
    StructField('id_transacao', IntegerType()),
    StructField('valor', DoubleType()),
    StructField('remetente', schema_remetente_destinatario),
    StructField('destinatario', schema_remetente_destinatario),
    StructField('chave_pix', StringType()),
    StructField('categoria', StringType()),
    StructField('transaction_date', StringType()),
    StructField('fraude', IntegerType())
])

caminho_json = 'case_final.json'

df = spark.read.json(
    caminho_json,
    schema=schema_base_pix,
    timestampFormat="yyyy-MM-dd HH:mm:ss"
)

df = df.withColumn(
      'destinatario_nome', col('destinatario').getField('nome')
    ).withColumn(
      'destinatario_banco', col('destinatario').getField('banco')
    ).withColumn(
      'destinatario_tipo', col('destinatario').getField('tipo')
    ).withColumn(
      'remetente_nome', col('remetente').getField('nome')
    ).withColumn(
      'remetente_banco', col('remetente').getField('banco')
    ).withColumn(
      'remetente_tipo', col('remetente').getField('tipo')
).drop('remetente', 'destinatario')

In [12]:
from pydeequ.analyzers import AnalysisRunner, AnalyzerContext, ApproxCountDistinct, Completeness, Compliance, Mean, Size


analysisResult = (
    AnalysisRunner(spark).onData(df)
    .addAnalyzer(Size())
    .addAnalyzer(Completeness('id_transacao'))
    .addAnalyzer(Compliance("valor", "valor > 0"))
    .run()
)


In [13]:
analysisResult

JavaObject id=o107

In [14]:
analysisResult_df = AnalyzerContext.successMetricsAsDataFrame(spark, analysisResult)


/home/valdir/Documentos/CURSO_DNC/novo-curso-dnc/dnc/lib/python3.8/site-packages/pyspark/sql/dataframe.py:127: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


In [15]:
analysisResult_df.show()

+-------+------------+------------+--------+
| entity|    instance|        name|   value|
+-------+------------+------------+--------+
|Dataset|           *|        Size|100000.0|
| Column|id_transacao|Completeness|     1.0|
| Column|       valor|  Compliance| 0.99972|
+-------+------------+------------+--------+



In [16]:
from pydeequ.suggestions import ConstraintSuggestionRunner, DEFAULT

suggestionResult = ConstraintSuggestionRunner(spark).onData(df).addConstraintRule(DEFAULT()).run()

23/10/24 23:35:46 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [18]:
for sugg in suggestionResult['constraint_suggestions']:
  print(f"Sugestao de Constraint: \'{sugg['column_name']}\': {sugg['description']}")
  print(f"PySpark Code: {sugg['code_for_constraint']}\n")

Sugestao de Constraint: 'destinatario_nome': 'destinatario_nome' is not null
PySpark Code: .isComplete("destinatario_nome")

Sugestao de Constraint: 'remetente_nome': 'remetente_nome' has value range 'Jonathan Gonsalves'
PySpark Code: .isContainedIn("remetente_nome", ["Jonathan Gonsalves"])

Sugestao de Constraint: 'remetente_nome': 'remetente_nome' is not null
PySpark Code: .isComplete("remetente_nome")

Sugestao de Constraint: 'id_transacao': 'id_transacao' is not null
PySpark Code: .isComplete("id_transacao")

Sugestao de Constraint: 'id_transacao': 'id_transacao' has no negative values
PySpark Code: .isNonNegative("id_transacao")

Sugestao de Constraint: 'id_transacao': 'id_transacao' is unique
PySpark Code: .isUnique("id_transacao")

Sugestao de Constraint: 'remetente_banco': 'remetente_banco' has value range 'BTG'
PySpark Code: .isContainedIn("remetente_banco", ["BTG"])

Sugestao de Constraint: 'remetente_banco': 'remetente_banco' is not null
PySpark Code: .isComplete("remetente_

In [19]:
from pydeequ.checks import Check, CheckLevel, ConstrainableDataTypes
from pydeequ.verification import VerificationResult, VerificationSuite

check = Check(spark, CheckLevel.Warning, "Review Check")
error = Check(spark, CheckLevel.Error, "Error")

In [22]:
checkResult = (
    VerificationSuite(spark)
      .onData(df)
      .addCheck(
        check.hasDataType("id_transacao",ConstrainableDataTypes.Integral)
        .isNonNegative("id_transacao")
        .isComplete("id_transacao") 
        .isUnique('id_transcao')
      )
  .run()
)

In [23]:
checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkResult)
checkResult_df.show(truncate=False)

+------------+-----------+------------+----------------------------------------------------------------------------------------------------------------------------+-----------------+-----------------------------------------------+
|check       |check_level|check_status|constraint                                                                                                                  |constraint_status|constraint_message                             |
+------------+-----------+------------+----------------------------------------------------------------------------------------------------------------------------+-----------------+-----------------------------------------------+
|Review Check|Warning    |Warning     |AnalysisBasedConstraint(DataType(id_transacao,None),<function1>,Some(<function1>),None)                                     |Success          |                                               |
|Review Check|Warning    |Warning     |ComplianceConstraint(Compliance(id_tr

/home/valdir/Documentos/CURSO_DNC/novo-curso-dnc/dnc/lib/python3.8/site-packages/pyspark/sql/dataframe.py:127: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


In [24]:
checkResult = (
    VerificationSuite(spark)
      .onData(df)
      .addCheck(
        error
          .isContainedIn("remetente_tipo", ["CNPJ"])
      )
  .run()
)

In [25]:
checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkResult)
checkResult_df.show(truncate=False)

+-----+-----------+------------+--------------------------------------------------------------------------------------------------------------------------------+-----------------+----------------------------------------------------+
|check|check_level|check_status|constraint                                                                                                                      |constraint_status|constraint_message                                  |
+-----+-----------+------------+--------------------------------------------------------------------------------------------------------------------------------+-----------------+----------------------------------------------------+
|Error|Error      |Error       |ComplianceConstraint(Compliance(remetente_tipo contained in CNPJ,`remetente_tipo` IS NULL OR `remetente_tipo` IN ('CNPJ'),None))|Failure          |Value: 0.0 does not meet the constraint requirement!|
+-----+-----------+------------+------------------------------------